In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 1. 데이터 불러오기
file_path = "/content/drive/MyDrive/통계데이터사이언스/lending_club/lending_club_processed2.csv"
df = pd.read_csv(file_path)

# 2. 종속변수, 독립변수 분리
y = df['default']
X = df.drop(columns=['default'])

# 3. 결과 저장용 리스트
auc_list = []
acc_list = []

# 4. 3회 반복 평가
for _ in tqdm(range(3), desc="Random Forest 3회 반복"):
    # 4-1. Train(60%) / Valid(20%) / Test(20%) 분할
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=None)
    X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=None)

    # 4-2. 모델 학습
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=None,
        random_state=None,
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # 4-3. 성능 평가 (Test셋 기준)
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred_prob)
    acc = accuracy_score(y_test, y_pred)

    # 4-4. 기록
    auc_list.append(auc)
    acc_list.append(acc)

# 5. 평균 및 표준편차 출력
print("\n최종 성능 요약 (Test set 기준)")
print(f"AUC 평균: {np.mean(auc_list):.4f} ± {np.std(auc_list):.4f}")
print(f"ACC 평균: {np.mean(acc_list):.4f} ± {np.std(acc_list):.4f}")


Random Forest 3회 반복: 100%|██████████| 3/3 [24:35<00:00, 492.00s/it]


최종 성능 요약 (Test set 기준)
AUC 평균: 0.9372 ± 0.0004
ACC 평균: 0.8976 ± 0.0004


In [4]:
# 1. 확률 기준 설정
threshold = 0.7

# 2. 투자 대상 선택
invest_idx = y_pred_prob > threshold
invest_results = y_test[invest_idx].reset_index(drop=True)

# 3. 수익률 계산: 성공(+1), 실패(-1)
returns = invest_results.apply(lambda x: 1 if x == 0 else -1)

# 4. Sharpe Ratio 계산
expected_return = returns.mean()
risk = returns.std()
sharpe_ratio = expected_return / risk if risk > 0 else 0

# 5. 결과 출력
print(f"투자 기준 확률: {threshold}")
print(f"총 투자 건수: {len(returns)}")
print(f"평균 수익률: {expected_return:.4f}")
print(f"수익률 표준편차: {risk:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")


투자 기준 확률: 0.7
총 투자 건수: 21278
평균 수익률: 0.6055
수익률 표준편차: 0.7959
Sharpe Ratio: 0.7608
